
## Introduction

Apprentice Chef, Inc. is a meal kit company. They offer gourmet meals that are daily-prepared to your doorstep. 

A cross-selling promotion called Halfway There has been launched by Apprentice Chef in order to diversify their revenue stream. 

Customers who subscribe to Halfway There will receive a half bottle of wine from a local California vineyard every Wednesday.

Apprentice Chef want to know which customers will subscribe to this service.


In [1]:
# importing libraries
import numpy                   as np                         # mathematical essentials
import pandas                  as pd                         # data science essentials
import matplotlib.pyplot       as plt                        # essential graphical output
import seaborn                 as sns                        # enhanced graphical output
import statsmodels.formula.api as smf                        # regression modeling
from sklearn.model_selection   import train_test_split       # train-test split
from sklearn.linear_model      import LogisticRegression     # logistic regression
from sklearn.metrics           import confusion_matrix       # confusion matrix
from sklearn.metrics           import roc_auc_score          # auc score
from sklearn.neighbors         import KNeighborsClassifier   # KNN for classification
from sklearn.neighbors         import KNeighborsRegressor    # KNN for regression
from sklearn.preprocessing     import StandardScaler         # standard scaler
from sklearn.tree              import DecisionTreeClassifier # classification trees
from sklearn.tree              import export_graphviz        # exports graphics
from sklearn.externals.six     import StringIO               # saves objects in memory
from IPython.display           import Image                  # displays on frontend



/Users/mansiyadav/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [2]:
# Read the file
file = 'Apprentice_Chef_Dataset.xlsx'

# Read the file as original_df
original_df = pd.read_excel(file)

# coping the new dataset
mydf = original_df

# View the top 5 columns of the dataset
#mydf.head(n = 5)

In [3]:
## Correlations with CROSS SELL SUCCESS
#df_corr = mydf.corr().round(2)
#df_corr.loc['CROSS_SELL_SUCCESS'].sort_values(ascending = False)

Checking to see what is the biggest factor in determining cross sell success. <br>

Followed recommendations pct: Percentage of time customer followed the meal recommendation generated for them. <br>
The more people follow recommendations the more likely they are to buy wine that we're trying to sell to them. (Cross sell success)


## Feature Engineering

Splitting email address into <br>
1. Personal
2. Professional
3. Junk

In [4]:
## create an empty list
sep_email = [] 

# looping over each email address
for index, col in mydf.iterrows(): 
    split_email = mydf.loc[index, 'EMAIL'].split(sep = '@') # splitting email domain at '@'
    sep_email.append(split_email)                           # appending sep_email with the results
    
# converting sep_email into a DataFrame 
email_df = pd.DataFrame(sep_email)

# Concatenating with original dataframe
# Reading the original file again
mydf = pd.read_excel('Apprentice_Chef_Dataset.xlsx')

# renaming column to concatenate
email_df.columns = ['NAME' , 'EMAIL_DOMAIN']

# concatenating personal_email_domain with original DataFrame 
mydf = pd.concat([mydf, email_df.loc[:, 'EMAIL_DOMAIN']],
                 axis = 1)

# EMAIL DOMAIN types
professional_email_domains = ['@mmm.com','@amex.com','@apple.com','@boeing.com','@caterpillar.com', 
                              '@chevron.com','@cisco.com','@cocacola.com','@disney.com','@dupont.com', 
                              '@exxon.com','@ge.org','@goldmansacs.com','@homedepot.com','@ibm.com',
                              '@intel.com','@jnj.com','@jpmorgan.com','@mcdonalds.com','@merck.com', 
                              '@microsoft.com','@nike.com','@pfizer.com','@pg.com','@travelers.com', 
                              '@unitedtech.com','@unitedhealth.com','@verizon.com','@visa.com','@walmart.com']
personal_email_domains     = ['@gmail.com', '@yahoo.com', '@protonmail.com']
junk_email_domains         = ['@me.com', '@aol.com', '@hotmail.com','@live.com', '@msn.com', '@passport.com']

# create another new empty list
new_lst = []  

# looping to group observations by domain type
for domain in mydf['EMAIL_DOMAIN']:
    if '@' + domain in professional_email_domains: 
        new_lst.append('professional')
    elif '@' + domain in personal_email_domains:
        new_lst.append('personal')
    elif '@' + domain in junk_email_domains:
        new_lst.append('junk')
    else:
        print('Unknown')

# concatenating with original DataFrame
mydf['DOMAIN_GRP'] = pd.Series(new_lst)

# checking results
mydf['DOMAIN_GRP'].value_counts()

personal        861
professional    696
junk            389
Name: DOMAIN_GRP, dtype: int64

In [5]:
# One hot encoding categorical variables
one_hot_DOMAIN_GRP = pd.get_dummies(mydf['DOMAIN_GRP'])

# Drop categorical variables after they've been encoded
mydf = mydf.drop('DOMAIN_GRP', axis = 1)

# Join codings together
mydf = mydf.join([one_hot_DOMAIN_GRP])

# Save new columns
new_columns = mydf.columns

Flagging outliers 

In [7]:
# flagging outliers
# setting outlier thresholds
TOTAL_MEALS_ORDERED_hi         = 300 
UNIQUE_MEALS_PURCH_hi          = 10
CONTACTS_W_CUSTOMER_SERVICE_lo = 3  
CONTACTS_W_CUSTOMER_SERVICE_hi = 12
AVG_TIME_PER_SITE_VISIT_hi     = 200 
CANCELLATIONS_BEFORE_NOON_hi   = 7    
CANCELLATIONS_AFTER_NOON_hi    = 2
WEEKLY_PLAN_hi                 = 15
LATE_DELIVERIES_hi             = 10     
AVG_PREP_VID_TIME_hi           = 300
LARGEST_ORDER_SIZE_lo          = 1
LARGEST_ORDER_SIZE_hi          = 10
AVG_CLICKS_PER_VISIT_lo        = 8
REVENUE_hi                     = 2500



##############################################################################
## Feature Engineering (outlier thresholds)                                 ##
##############################################################################

# developing features (columns) for outliers

# Total_meals_ordered
mydf['OUT_TOTAL_MEALS_ORDERED'] = 0
condition_hi = mydf.loc[0:,'OUT_TOTAL_MEALS_ORDERED'][mydf['TOTAL_MEALS_ORDERED'] > TOTAL_MEALS_ORDERED_hi]

mydf['OUT_TOTAL_MEALS_ORDERED'].replace(to_replace = condition_hi,
                                        value      = 1, 
                                        inplace    = True)

# Unique_meals_ordered
mydf['OUT_UNIQUE_MEALS_PURCH'] = 0
condition_hi = mydf.loc[0:, 'OUT_UNIQUE_MEALS_PURCH'][mydf['UNIQUE_MEALS_PURCH'] > UNIQUE_MEALS_PURCH_hi] 

mydf['OUT_UNIQUE_MEALS_PURCH'].replace(to_replace = condition_hi,
                                       value      = 1,
                                       inplace     = True)

# Contacts_W_customer_service
mydf['OUT_CONTACTS_W_CUSTOMER_SERVICE'] = 0
condition_lo = mydf.loc[0:, 'OUT_CONTACTS_W_CUSTOMER_SERVICE'][mydf['CONTACTS_W_CUSTOMER_SERVICE'] < CONTACTS_W_CUSTOMER_SERVICE_lo] 

mydf['OUT_UNIQUE_MEALS_PURCH'].replace(to_replace = condition_lo,
                                       value      = 1,
                                       inplace    = True)

# contacts_w_customer_service
mydf['OUT_CONTACTS_W_CUSTOMER_SERVICE'] = 0
condition_hi = mydf.loc[0:,'OUT_CONTACTS_W_CUSTOMER_SERVICE'][mydf['CONTACTS_W_CUSTOMER_SERVICE'] > CONTACTS_W_CUSTOMER_SERVICE_hi]

mydf['OUT_CONTACTS_W_CUSTOMER_SERVICE'].replace(to_replace = condition_hi,
                                                value      = 1, 
                                                inplace    = True)


# avg_time_per_site_visit
mydf['OUT_AVG_TIME_PER_SITE_VISIT'] = 0
condition_hi = mydf.loc[0:,'OUT_AVG_TIME_PER_SITE_VISIT'][mydf['AVG_TIME_PER_SITE_VISIT'] > AVG_TIME_PER_SITE_VISIT_hi]

mydf['OUT_AVG_TIME_PER_SITE_VISIT'].replace(to_replace = condition_hi,
                                            value      = 1, 
                                            inplace    = True)


# cancellations_before_noon
mydf['OUT_CANCELLATIONS_BEFORE_NOON'] = 0
condition_hi = mydf.loc[0:,'OUT_CANCELLATIONS_BEFORE_NOON'][mydf['CANCELLATIONS_BEFORE_NOON'] > CANCELLATIONS_BEFORE_NOON_hi]

mydf['OUT_CANCELLATIONS_BEFORE_NOON'].replace(to_replace = condition_hi,
                                              value      = 1, 
                                              inplace    = True)


# cancellations_after_noon
mydf['OUT_CANCELLATIONS_AFTER_NOON'] = 0
condition_hi = mydf.loc[0:,'OUT_CANCELLATIONS_AFTER_NOON'][mydf['CANCELLATIONS_AFTER_NOON'] > CANCELLATIONS_AFTER_NOON_hi]

mydf['OUT_CANCELLATIONS_AFTER_NOON'].replace(to_replace = condition_hi,
                                             value      = 1, 
                                             inplace    = True)


# weekly_plan
mydf['OUT_WEEKLY_PLAN'] = 0
condition_hi = mydf.loc[0:,'OUT_WEEKLY_PLAN'][mydf['WEEKLY_PLAN'] > WEEKLY_PLAN_hi]

mydf['OUT_WEEKLY_PLAN'].replace(to_replace = condition_hi,
                                value      = 1, 
                                inplace    = True)

# late_deliveries
mydf['OUT_LATE_DELIVERIES'] = 0
condition_hi = mydf.loc[0:,'OUT_LATE_DELIVERIES'][mydf['LATE_DELIVERIES'] > LATE_DELIVERIES_hi]

mydf['OUT_LATE_DELIVERIES'].replace(to_replace = condition_hi,
                                    value      = 1, 
                                    inplace    = True)

# avg_prep_vid_time
mydf['OUT_AVG_PREP_VID_TIME'] = 0
condition_hi = mydf.loc[0:,'OUT_AVG_PREP_VID_TIME'][mydf['AVG_PREP_VID_TIME'] > AVG_PREP_VID_TIME_hi]

mydf['OUT_AVG_PREP_VID_TIME'].replace(to_replace = condition_hi,
                                      value      = 1, 
                                      inplace    = True)


# largest_order_size
mydf['OUT_LARGEST_ORDER_SIZE'] = 0
condition_hi = mydf.loc[0:,'OUT_LARGEST_ORDER_SIZE'][mydf['LARGEST_ORDER_SIZE'] > LARGEST_ORDER_SIZE_hi]
condition_lo = mydf.loc[0:,'OUT_LARGEST_ORDER_SIZE'][mydf['LARGEST_ORDER_SIZE'] < LARGEST_ORDER_SIZE_lo]
mydf['OUT_LARGEST_ORDER_SIZE'].replace(to_replace = condition_lo,
                                       value      = 1, 
                                       inplace    = True)
mydf['OUT_LARGEST_ORDER_SIZE'].replace(to_replace = condition_hi,
                                       value      = 1, 
                                       inplace    = True)


# avg_clicks_per_visit
mydf['OUT_AVG_CLICKS_PER_VISIT'] = 0
condition_lo = mydf.loc[0:,'OUT_AVG_CLICKS_PER_VISIT'][mydf['AVG_CLICKS_PER_VISIT'] < AVG_CLICKS_PER_VISIT_lo]

mydf['OUT_AVG_CLICKS_PER_VISIT'].replace(to_replace = condition_lo,
                                         value      = 1, 
                                         inplace    = True)


# revenue
mydf['out_REVENUE'] = 0
condition_hi = mydf.loc[0:,'out_REVENUE'][mydf['REVENUE'] > REVENUE_hi]

mydf['out_REVENUE'].replace(to_replace = condition_hi,
                            value      = 1,
                            inplace    = True)

## Building Models

In [8]:
# Explanatory Variables
mydf_data = mydf.drop(['CROSS_SELL_SUCCESS','NAME' , 'EMAIL_DOMAIN','EMAIL','FIRST_NAME', 'FAMILY_NAME'], axis=1)

# Response Variable
mydf_target = mydf.loc[:, 'CROSS_SELL_SUCCESS']

#for val in mydf_data:
#    print(f"{val} +")

In [9]:
# instantiating a logistic regression model object
logit_sig = smf.logit(formula = """ CROSS_SELL_SUCCESS ~  MOBILE_NUMBER +
                                                            CANCELLATIONS_BEFORE_NOON +
                                                            CANCELLATIONS_AFTER_NOON +
                                                            TASTES_AND_PREFERENCES +
                                                            PC_LOGINS +
                                                            MOBILE_LOGINS +
                                                            REFRIGERATED_LOCKER +
                                                            FOLLOWED_RECOMMENDATIONS_PCT +
                                                            professional +
                                                            OUT_UNIQUE_MEALS_PURCH +
                                                            OUT_AVG_PREP_VID_TIME """,
                                     data    = mydf)


# fitting the model object
logit_sig = logit_sig.fit()


# checking the results SUMMARY
logit_sig.summary()

Optimization terminated successfully.
         Current function value: 0.452628
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     CROSS_SELL_SUCCESS   No. Observations:                 1946
Model:                          Logit   Df Residuals:                     1934
Method:                           MLE   Df Model:                           11
Date:                Sun, 15 Mar 2020   Pseudo R-squ.:                  0.2790
Time:                        19:51:08   Log-Likelihood:                -880.81
converged:                       True   LL-Null:                       -1221.6
Covariance Type:            nonrobust   LLR p-value:                4.800e-139
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -3.2552      0.625     -5.211      0.000      -4.479      -2.031
MOBILE_NUMBER                    0.7259      0.174      4.182      0.000       0.386       1.066
CANCELLATIONS_BEFORE_NOON        0.2518      0.043      5.865      0.000       0.168       0.336
CANCELLATIONS_AFTER_NOON        -0.2893      0.132     -2.197      0.028      -0.547      -0.031
TASTES_AND_PREFERENCES           0.4552      0.128      3.560      0.000       0.205       0.706
PC_LOGINS                        0.2243      0.101      2.214      0.027       0.026       0.423
MOBILE_LOGINS                   -0.2960      0.111     -2.667      0.008      -0.514      -0.078
REFRIGERATED_LOCKER              0.5065      0.200      2.535      0.011       0.115       0.898
FOLLOWED_RECOMMENDATIONS_PCT     0.0571      0.003     16.961      0.000       0.050       0.064
professional                     1.0790      0.131      8.238      0.000       0.822       1.336
OUT_UNIQUE_MEALS_PURCH          -1.1385      0.558     -2.040      0.041      -2.232      -0.045
OUT_AVG_PREP_VID_TIME            1.5391      0.701      2.197      0.028       0.166       2.912
================================================================================================
"""

Here we can see the significant variables in our logistic regression model. 

We will now use these significant variables to train and test our data to build different models. 

#### Data Prep:
Train<br>
Test<br>
Split<br>


In [10]:
# declaring the x variables
x_variables = ['MOBILE_NUMBER', 'CANCELLATIONS_BEFORE_NOON',
               'CANCELLATIONS_AFTER_NOON', 'TASTES_AND_PREFERENCES',
               'PC_LOGINS', 'MOBILE_LOGINS', 'REFRIGERATED_LOCKER',
               'FOLLOWED_RECOMMENDATIONS_PCT', 'professional',
               'OUT_UNIQUE_MEALS_PURCH', 'OUT_AVG_PREP_VID_TIME']

# train/test split with the full model
mydf_data   =  mydf.loc[ : , x_variables]
mydf_target =  mydf.loc[ : , 'CROSS_SELL_SUCCESS']

# train test split
X_train, X_test, y_train, y_test = train_test_split(
            mydf_data,
            mydf_target,
            test_size    = 0.25,
            random_state = 222,
            stratify     = mydf_target)

### Model 1
KNN Classification

In [11]:
# create lists for training set accuracy and test set accuracy
training_accuracy = []
test_accuracy = []


# build a visualization 
neighbors_settings = range(1, 18)


for n_neighbors in neighbors_settings:
    clf = KNeighborsClassifier(n_neighbors = n_neighbors) # building the model
    clf.fit(X_train, y_train)
    training_accuracy.append(clf.score(X_train, y_train)) # recording the training set accuracy
    test_accuracy.append(clf.score(X_test, y_test))       # recording the generalization accuracy


# plot the visualization
#fig, ax = plt.subplots(figsize=(12,8))
#plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
#plt.plot(neighbors_settings, test_accuracy, label = "test accuracy")
#plt.ylabel("Accuracy")
#plt.xlabel("n_neighbors")
#plt.legend()
#plt.show()

# find the optimal number of neighbors
opt_neighbors = test_accuracy.index(max(test_accuracy)) + 1
#print(f"""The optimal number of neighbors is {opt_neighbors}""")

In [12]:
# Instantiating StandardScaler()
scaler = StandardScaler()

# Fitting the data
scaler.fit(mydf_data)

# Transforming the data
X_scaled     = scaler.transform(mydf_data)

# converting to a DataFrame
X_scaled_df  = pd.DataFrame(X_scaled) 

# train-test split with the scaled data
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
            X_scaled_df,
            mydf_target,
            random_state = 222,
            test_size = 0.25,
            stratify = mydf_target)

# Instantiating a KNN classification model with optimal neighbors
knn_opt = KNeighborsClassifier(n_neighbors = opt_neighbors)


# Fitting the training data
knn_fit = knn_opt.fit(X_train_scaled, y_train_scaled)


# Predicting based on the testing set
knn_pred = knn_fit.predict(X_test_scaled)


# Scoring the results
#print('Training ACCURACY:', knn_fit.score(X_train_scaled, y_train_scaled).round(4))
#print('Testing  ACCURACY:', knn_fit.score(X_test_scaled, y_test_scaled).round(4))
#print('AUC Score        :', roc_auc_score(y_true  = y_test_scaled,
#                                          y_score = knn_pred).round(4))


# creating an empty list for the results of all models
model_performance = [['Model', 'Training Accuracy',
                      'Testing Accuracy', 'AUC Value']]
# Train accuracy
knn_train_acc = knn_fit.score(X_train_scaled, y_train_scaled).round(4)


# Test accuracy
knn_test_acc  = knn_fit.score(X_test_scaled, y_test_scaled).round(4)


# auc value
knn_auc       = roc_auc_score(y_true  = y_test_scaled,
                              y_score = knn_pred).round(4)


# Saving the results
model_performance.append(['KNN Classification',
                          knn_train_acc,
                          knn_test_acc,
                          knn_auc])

### Model 2 
Logistic Regression Model

In [13]:
# Instantiating a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 1,
                            random_state = 567)


# Fitting the training data
logreg_fit = logreg.fit(X_train, y_train)


# Predicting based on the testing set
logreg_pred = logreg_fit.predict(X_test)


# Scoring the results
#print('Training ACCURACY:', logreg_fit.score(X_train, y_train).round(4))
#print('Testing  ACCURACY:', logreg_fit.score(X_test, y_test).round(4))
#print('AUC Score        :', roc_auc_score(y_true  = y_test,
#                                          y_score = logreg_pred).round(4))

# train accuracy
logreg_train_acc  = logreg_fit.score(X_train, y_train).round(4)

# test accuracy
logreg_test_acc   = logreg_fit.score(X_test, y_test).round(4)

# auc value
logreg_auc = roc_auc_score(y_true  = y_test,
                           y_score = logreg_pred).round(4)

# saving the results
model_performance.append(['Logistic Regression',
                          logreg_train_acc,
                          logreg_test_acc,
                          logreg_auc])



/Users/mansiyadav/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Model 3
Full Tree

In [14]:
# Instantiating a classification tree object
full_tree = DecisionTreeClassifier()


# Fitting the training data
full_tree_fit = full_tree.fit(X_train, y_train)


# Predicting on new data
full_tree_pred = full_tree_fit.predict(X_test)

# Scoring the model 
#print('Training ACCURACY:', full_tree_fit.score(X_train, y_train).round(4))
#print('Testing  ACCURACY:', full_tree_fit.score(X_test, y_test).round(4))
#print('AUC Score        :', roc_auc_score(y_true  = y_test,
#                                          y_score = full_tree_pred).round(4))

# Train accuracy
full_tree_train_acc = full_tree_fit.score(X_train, y_train).round(4)

# Test accuracy
full_tree_test_acc  = full_tree_fit.score(X_test, y_test).round(4)

# auc value
full_tree_auc       = roc_auc_score(y_true  = y_test,
                                    y_score = full_tree_pred).round(4)

# Saving the results
model_performance.append(['Full Tree',
                          full_tree_train_acc,
                          full_tree_test_acc,
                          full_tree_auc])


### Model 4
Pruned Tree

In [15]:
# Instantiaing a classification tree object
tree_pruned      = DecisionTreeClassifier(max_depth = 4,
                                          min_samples_leaf = 25,
                                          random_state = 222)


# Fitting the training data
tree_pruned_fit  = tree_pruned.fit(X_train, y_train)


# Predicting on new data
tree_pred = tree_pruned_fit.predict(X_test)


# Scoring the model
#print('Training ACCURACY:', tree_pruned_fit.score(X_train, y_train).round(4))
#print('Testing  ACCURACY:', tree_pruned_fit.score(X_test, y_test).round(4))
#print('AUC Score        :', roc_auc_score(y_true  = y_test,
#                                          y_score = tree_pred).round(4))

# Train accuracy
p_tree_train_acc = tree_pruned_fit.score(X_train, y_train).round(4)


# Test accuracy
p_tree_test_acc  = tree_pruned_fit.score(X_test, y_test).round(4)


# auc value
p_tree_auc       = roc_auc_score(y_true  = y_test,
                                 y_score = tree_pred).round(4)

# Saving the results
model_performance.append(['Pruned Tree',
                          p_tree_train_acc,
                          p_tree_test_acc,
                          p_tree_auc])

## Final Results

In [16]:
# converting to DataFrame and checking the results
pd.DataFrame(model_performance[1:], columns = model_performance[0])

,Model,Training Accuracy,Testing Accuracy,AUC Value
0,KNN Classification,0.7964,0.7659,0.7600
1,Logistic Regression,0.7498,0.7474,0.7108
2,Full Tree,0.9136,0.7207,0.7064
3,Pruned Tree,0.8095,0.7947,0.7574


In [17]:
# declaring a DataFrame object
model_performance_df = pd.DataFrame(model_performance[1:], columns = model_performance[0])


# saving the DataFrame to Excel
model_performance_df.to_excel('Classification Model Performance.xlsx',
                              index = False)